In [1]:
    !which python; python -V;

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


In [2]:
from poliastro.bodies import Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Sun, Pluto
from poliastro.twobody import Orbit

from poliastro.constants import GM_earth

from astropy import units as u
import numpy as np

from perylune.orbit_tools import *
from perylune.interplanetary import *
from perylune.constants import *

In [11]:
# Let's create two departure orbits. Typically the departure orbit is circular (orb1), but we can make some
# interesting observations on eliptical orbit (orb2) and study how the escape velocity changes between apoapsis and periapsis.
orb1 = Orbit.circular(Earth, 250*u.km)
orb2 = Orbit.from_classical(Earth, 350*u.km + Earth.R, 0.14285714285714285*u.one, 0*u.deg, 0*u.deg, 0*u.deg, 0*u.deg)

In [12]:
print_orb(orb1)
print_orb(orb2)

6628 x 6628 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)
a(𝑎)=6628.14km, b=6628.14km, e=0.00, i=0.00deg raan(Ω)=0.00deg argp(𝜔)=0.00deg nu(𝜈)=0.00deg
period=5370.30s perapis=6628km(250.00km) apoapsis=6628km(250.00km)
5767 x 7689 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)
a(𝑎)=6728.14km, b=6659.13km, e=0.14, i=0.00deg raan(Ω)=0.00deg argp(𝜔)=0.00deg nu(𝜈)=0.00deg
period=5492.29s perapis=5767km(-611.16km) apoapsis=7689km(1311.16km)


In [13]:
# Calculate delta-v necessary to reach escape velocity for circular orbit. The values returned are current position, periapsis, apoapis.
# No surprises here - it's perfectly circular, so it's always the same.
escape_delta_v(orb1, False)

(<Quantity 3212.16227604 m / s>,
 <Quantity 3212.16227604 m / s>,
 <Quantity 3212.16227604 m / s>)

In [6]:
# The situation for eliptical orbit is more interesting. The delta-v for periapsis burn is 2869 m/s, which is the smallest value
# for this orbit and up to 3516 for apoapsis (when the velocity is smallest, so extra delta-v is needed to escape)
escape_delta_v(orb2, False)

(<Quantity 2869.63167184 m / s>,
 <Quantity 2869.63167184 m / s>,
 <Quantity 3516.37639716 m / s>)

In [7]:
# Here's an utility function that returns a (semi-major axis) and e (eccentricity) for min and max altitude orbits
apses_to_ae(400, 300)
apses_to_ae(300*u.km , 400*u.km, Earth.R)

(350.0, 0.14285714285714285)